## Illustrate the usage of Sympy. Rotation Matrexes and Kinematics

In [1]:
# Import the required modules
from sympy import symbols, cos, sin, pi, simplify, atan2, sqrt
from sympy.matrices import Matrix
import numpy as np

In [2]:
# Conversion Factors
rtd = 180./np.pi # radians to degrees
dtr = np.pi/180. # degrees to radian

** Rotation Matrixes in 3D Space **

In [3]:
## Build the rotation matrix versus the x, y and z axes
def R_x(q):
    return Matrix([[ 1,      0,       0],
                   [ 0, cos(q), -sin(q)],
                   [ 0, sin(q),  cos(q)]])    
    
def R_y(q):
    return Matrix([[  cos(q), 0, sin(q)],
                   [       0, 1,      0],
                   [ -sin(q), 0, cos(q)]])
        
def R_z(q):
    return Matrix([[ cos(q), -sin(q), 0],
                   [ sin(q),  cos(q), 0],
                   [      0,       0, 1]])

In [4]:
## Evaluate the matrix for a rotation on the x axes by 45degree
print("Rotation about the X-axis by 45-degrees")
print(R_x(45*dtr))

Rotation about the X-axis by 45-degrees
Matrix([[1, 0, 0], [0, 0.707106781186548, -0.707106781186547], [0, 0.707106781186547, 0.707106781186548]])


In [5]:
print("Rotation about the y-axis by 45-degrees")
print(R_y(45*dtr))

Rotation about the y-axis by 45-degrees
Matrix([[0.707106781186548, 0, 0.707106781186547], [0, 1, 0], [-0.707106781186547, 0, 0.707106781186548]])


In [6]:
print("Rotation about the Z-axis by 30-degrees")
print(R_z(30*dtr))

Rotation about the Z-axis by 30-degrees
Matrix([[0.866025403784439, -0.500000000000000, 0], [0.500000000000000, 0.866025403784439, 0], [0, 0, 1]])


** Combined Rotation Matrix. Note! that the sequence is important as the end result is not the same **

In [7]:
R_xyz = simplify(R_x(45*dtr) * R_y(45*dtr) * R_z(30*dtr))
R_zyx = simplify(R_z(30*dtr) * R_y(45*dtr) * R_x(45*dtr))
R_yxz = simplify(R_y(45*dtr) * R_x(45*dtr) * R_z(30*dtr))

print(R_xyz)
print(R_zyx)
print(R_yxz)

Matrix([[0.612372435695795, -0.353553390593274, 0.707106781186547], [0.786566092485493, 0.362372435695795, -0.500000000000000], [-0.0794593112989457, 0.862372435695794, 0.500000000000000]])
Matrix([[0.612372435695795, 0.0794593112989455, 0.786566092485493], [0.353553390593274, 0.862372435695794, -0.362372435695795], [-0.707106781186547, 0.500000000000000, 0.500000000000000]])
Matrix([[0.862372435695794, 0.0794593112989455, 0.500000000000000], [0.353553390593274, 0.612372435695795, -0.707106781186547], [-0.362372435695794, 0.786566092485493, 0.500000000000000]])


** Calculate Roll, Pitch and Yaw angle given a rotation matrix **

In [8]:
# Fixed Axis X-Y-Z Given Rotation Matrix
R_XYZ = Matrix([[ 0.353553390593274, -0.306186217847897, 0.883883476483184],
                [ 0.353553390593274,  0.918558653543692, 0.176776695296637],
                [-0.866025403784439,               0.25, 0.433012701892219]])

In [9]:
# calculate the paramters so to enable the composit rotation to get to the final position as per the above matrix.
r31 = R_XYZ[2,0]
r11 = R_XYZ[0,0]
r21 = R_XYZ[1,0]
r32 = R_XYZ[2,1]
r33 = R_XYZ[2,2]

In [10]:
### Euler Angles from Rotation Matrix
beta  = atan2(-r31, sqrt(r11 * r11 + r21 * r21)) * rtd
gamma = atan2(r32, r33) * rtd
alpha = atan2(r21, r11) * rtd

In [14]:
R_XYZ[2,1]

0.250000000000000

## Homogeneus Transform

In [11]:
# Calculate the position of a point P, which is known with respect to a frame B. Frame B is relative to the frame A. 
# Construct P in {B} represent the translation
P = Matrix([[15.0],[0.0],[42.0],[1]])

In [13]:
# Define Homogeneous Transform. _Note that this is only for a rotation in the Y axes.
qy = symbols('qy')
T = Matrix([[ cos(qy),   0,  sin(qy),    1.],
            [ 0,         1,        0,    0.],
            [ -sin(qy),  0,  cos(qy),   30.], 
            [ 0,       0,          0,   1 ]])

In [14]:
# Calculate new coordinates of P in {A}
P_inA = simplify(T * P)
print("P_new is :", P_inA)

P_new is : Matrix([[42.0*sin(qy) + 15.0*cos(qy) + 1.0], [0], [-15.0*sin(qy) + 42.0*cos(qy) + 30.0], [1]])


In [15]:
# Evaluate numerically
print("The new coordinates of P_inA are :", P_inA.evalf(subs={qy: 110*dtr}))

The new coordinates of P_inA are : Matrix([[35.3367879231231], [0], [1.53976466853329], [1.00000000000000]])


** DH parameters (Example of SCARA manipulator)**

In [16]:
### Create symbols for joint variables
# angles between X axis
theta1, theta2, theta3, theta4 = symbols('theta1:5')
# distances between X axis
d1, d2, d3, d4 = symbols('d1:5')
# distances between Z axis
a0, a1, a2, a3 = symbols('a0:4')
# angles between Z-axes
alpha0, alpha1, alpha2, alpha3 = symbols('alpha0:4')

In [17]:
# DH Parameters Table, described as a dictionary for the SCARA, with known values assigned
DH ={alpha0: 0,  a0:    0, d1:  0, theta1: theta1,
     alpha1: 0,  a1: 0.45, d2:  0, theta2: theta2,
     alpha2: 0,  a2: 0.30, d2: d2, theta3:      0,
     alpha3: 0,  a3:    0, d4:  0, theta4: theta4
    }

** Generalized Homogeneous Transform between 2 links **

In [18]:
def ghmt(alpha, a, d, theta, dh):
    HTM =  Matrix([[            cos(theta),           -sin(theta),           0,             a],
                   [ sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d],
                   [ sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  cos(alpha)*d],
                   [                     0,                      0,          0,             1]])
    return HTM.subs(dh)

In [19]:
#### Homogeneous Transforms fo all the link pairs
T0_1 = ghmt(alpha0, a0, d1, theta1, DH)
T1_2 = ghmt(alpha1, a1, d2, theta2, DH)
T2_3 = ghmt(alpha2, a2, d3, theta3, DH)
T3_4 = ghmt(alpha3, a3, d4, theta4, DH)

In [20]:
# Transform from base link to end effector
T0_4 = simplify(T0_1 * T1_2 * T2_3 * T3_4)
print(T0_4)

Matrix([[cos(theta1 + theta2 + theta4), -sin(theta1 + theta2 + theta4), 0, 0.45*cos(theta1) + 0.3*cos(theta1 + theta2)], [sin(theta1 + theta2 + theta4), cos(theta1 + theta2 + theta4), 0, 0.45*sin(theta1) + 0.3*sin(theta1 + theta2)], [0, 0, 1, d2 + d3], [0, 0, 0, 1]])


In [21]:
# Evaluate now for particular values
print(T0_4.evalf(subs={theta1: 0, theta2: 0, d3: -0.5, theta4: 0}))

Matrix([[1.00000000000000, 0, 0, 0.750000000000000], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, d2 + d3], [0, 0, 0, 1.00000000000000]])


## Project Code

In [22]:
### Create symbols for joint variables
# angles between X axis
theta1, theta2, theta3, theta4, theta5, theta6, theta7 = symbols('theta1:8')
# distances between X axis
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
# distances between Z axis
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
# angles between Z-axes
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

In [23]:
# DH Parameters Table, described as a dictionary for the SCARA, with known values assigned
DH ={alpha0:     0,  a0:      0, d1:  0.75, theta1:      theta1,
     alpha1: -pi/2,  a1:   0.35, d2:     0, theta2: theta2-pi/2, 
     alpha2:     0,  a2:   1.25, d3:     0, theta3:      theta3,
     alpha3: -pi/2,  a3: -0.054, d4:  1.50, theta4:      theta4,
     alpha4: -pi/2,  a4:      0, d5:     0, theta5:      theta5,
     alpha5: -pi/2,  a5:      0, d6:     0, theta6:      theta6,
     alpha6:     0,  a6:      0, d7: 0.303, theta7:           0
    }

In [24]:
# Homogeneous Transforms fo all the link pairs
T0_1 = ghmt(alpha0, a0, d1, theta1, DH)
T1_2 = ghmt(alpha1, a1, d2, theta2, DH)
T2_3 = ghmt(alpha2, a2, d3, theta3, DH)
T3_4 = ghmt(alpha3, a3, d4, theta4, DH)
T4_5 = ghmt(alpha4, a4, d5, theta5, DH)
T5_6 = ghmt(alpha5, a5, d6, theta6, DH)
T6_E = ghmt(alpha6, a6, d7, theta7, DH)

In [25]:
# Incrementally Build the homogeneous transforms between base link and the grip
T0_E = simplify(T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_E)
print(T0_E)

Matrix([[((sin(theta1)*sin(theta4) + sin(theta2 + theta3)*cos(theta1)*cos(theta4))*cos(theta5) - sin(theta5)*cos(theta1)*cos(theta2 + theta3))*cos(theta6) + (-sin(theta1)*cos(theta4) + sin(theta4)*sin(theta2 + theta3)*cos(theta1))*sin(theta6), -((sin(theta1)*sin(theta4) + sin(theta2 + theta3)*cos(theta1)*cos(theta4))*cos(theta5) - sin(theta5)*cos(theta1)*cos(theta2 + theta3))*sin(theta6) - (sin(theta1)*cos(theta4) - sin(theta4)*sin(theta2 + theta3)*cos(theta1))*cos(theta6), -(sin(theta1)*sin(theta4) + sin(theta2 + theta3)*cos(theta1)*cos(theta4))*sin(theta5) - cos(theta1)*cos(theta5)*cos(theta2 + theta3), -0.303*sin(theta1)*sin(theta4)*sin(theta5) + 1.25*sin(theta2)*cos(theta1) - 0.303*sin(theta5)*sin(theta2 + theta3)*cos(theta1)*cos(theta4) - 0.054*sin(theta2 + theta3)*cos(theta1) - 0.303*cos(theta1)*cos(theta5)*cos(theta2 + theta3) + 1.5*cos(theta1)*cos(theta2 + theta3) + 0.35*cos(theta1)], [((sin(theta1)*sin(theta2 + theta3)*cos(theta4) - sin(theta4)*cos(theta1))*cos(theta5) - sin(t

In [26]:
# Evaluate now for particular values
print(T0_E.evalf(subs={theta1: 0, theta2: 0, theta3: 0, theta4: 0, theta5: 0, theta6: 0, theta7: 0}))

Matrix([[0, 0, -1.00000000000000, 1.54700000000000], [0, 1.00000000000000, 0, 0], [1.00000000000000, 0, 0, 1.94600000000000], [0, 0, 0, 1.00000000000000]])


In [ ]:
# Correction needed to account for the differences in orientation between the DH format and the URDF description of the grip
T_y = Matrix([[ cos(-np.pi/2),        0,  sin(-np.pi/2),  0],
              [             0,        1,              0,  0],
              [-sin(-np.pi/2),        0,  cos(-np.pi/2),  0],
              [             0,        0,              0,  1]])

T_z = Matrix([[ cos(np.pi), -sin(np.pi),        0,  0],
              [ sin(np.pi),  cos(np.pi),        0,  0],
              [         0,            0,        1,  0],
              [         0,            0,        0,  1]])

T_Corr = simplify(T_z * T_y)

In [ ]:
# Total homogeneous transform
T_Total = simplify(T0_7 * T_Corr)
print(T_Total)

In [ ]:
# Evaluate now for particular values
print(T_Total.evalf(subs={theta1: 2.72, theta2: -0.41, theta3: -0.69, theta4: -1.28, theta5: 0.45, theta6: -1.04, theta7: 0}))

Matrix([[  -0.917120822816605,  4.88155637626401e-17,    -0.398609327984423, 0.971056868642107],
        [7.49879891330929e-33,     -1.00000000000000, -1.22464679914735e-16,                 0],
        [  -0.398609327984423, -1.12314908009374e-16,     0.917120822816605,  2.32401186968601],
        [                   0,                     0,                     0,  1.00000000000000]])